In [ ]:
!pip install diffusers transformers accelerate datasets torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel RGB
    transforms.Resize((512, 512)),                # Resize to 512x512 pixels
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])
mnist_dataset = datasets.MNIST(root="mnist_data", train=True, download=True, transform=transform)
mnist_dataloader = DataLoader(mnist_dataset, batch_size=16, shuffle=True)

In [ ]:
from diffusers import StableDiffusionPipeline, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import torch

# Load pre-trained Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe.to("cuda")

# Replace scheduler with DDPMScheduler for training
pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)

# Extract components
unet = pipe.unet
vae = pipe.vae
text_encoder = pipe.text_encoder
tokenizer = pipe.tokenizer

# Optimizer
optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)

# Training loop
num_epochs = 1
device = "cuda"
unet.train()

for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(mnist_dataloader):
        # Prepare images
        pixel_values = images.to(device)

        # Encode images to latents
        latents = vae.encode(pixel_values).latent_dist.sample()
        latents = latents * vae.config.scaling_factor

        # Add noise to latents
        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, pipe.scheduler.num_train_timesteps, (latents.size(0),), device=device).long()
        noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)

        # Prepare text embeddings
        prompts = [f"a photo of the number {label}" for label in labels]
        inputs = tokenizer(prompts, return_tensors="pt", padding="max_length", truncation=True, max_length=77)
        input_ids = inputs["input_ids"].to(device)
        encoder_hidden_states = text_encoder(input_ids).last_hidden_state

        # Predict noise
        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

        # Compute loss
        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 100 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.item()}")

# Save the fine-tuned model
pipe.save_pretrained("stable_diffusion_mnist")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacity of 14.75 GiB of which 245.06 MiB is free. Process 10595 has 14.51 GiB memory in use. Of the allocated memory 13.51 GiB is allocated by PyTorch, and 895.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from diffusers import StableDiffusionPipeline

# Load the fine-tuned model
pipe = StableDiffusionPipeline.from_pretrained("stable_diffusion_mnist")
pipe.to("cuda")

# Generate a new MNIST-style image
prompt = "a handwritten digit"
image = pipe(prompt).images[0]

# Display the image
image.show()